Here we compare 4 testing schemes: Dubey & Muller, Energy statistic (Szekely & Rizzo 2004), HIPM, WoW.

In particular we compare power functions on the example of random probability measures from Fig 1 Dubey & Muller.

In [ ]:
using Plots
using KernelDensity


include("distributions.jl")

include("distances/new_distance.jl")
include("distances/distance_Wasserstein.jl")


In [ ]:
function test_statistic_dm(pms_1::Vector{Normal}, pms_2::Vector{Normal})
    # given two samples of probobability measures, compute the test statistic according to Dubey & Muller
    n = length(pms_1)
    # Recall that we assume that each of the sample is Normal distribution, and for W_2 metric we only need their means.
    X = [pms_1[i].μ for i in 1:n] # collect all the means from first samples of probability measures
    Y = [pms_2[i].μ for i in 1:n] # collect all the means from second samples of probability measures
    
    return test_statistic_dm(X, Y)
end

function test_statistic_dm(μ_1::Vector{Float64}, μ_2::Vector{Float64})
    # given two samples of means of normal distributions, compute the test statistic according to Dubey & Muller
    n = length(μ_1)
    # Recall that we assume that each of the sample is Normal distribution, and for W_2 metric we only need their means.
    X = μ_1 # collect all the means from first samples of probability measures
    Y = μ_2 # collect all the means from second samples of probability measures

    μ_hat_1 = mean(X)
    μ_hat_2 = mean(Y)

    v_hat_1 = mean((X .- μ_hat_1).^2)
    v_hat_2 = mean((Y .- μ_hat_2).^2)

    σ_hat_1_squared = mean((X .- μ_hat_1).^4) - (v_hat_1)^2
    σ_hat_2_squared = mean((Y .- μ_hat_2).^4) - (v_hat_2)^2
    
    μ_hat_p = sum(X .+ Y) / (2 * n)
    v_hat_p = sum( (X .- μ_hat_p).^2 .+ (Y .- μ_hat_p).^2 ) / (2 * n)

    F_n = v_hat_p - v_hat_1/2 - v_hat_2/2
    U_n = (1/4) * ((v_hat_1 - v_hat_2)^2) / (σ_hat_1_squared * σ_hat_2_squared) 
    
    T_n = 2*n*U_n / (1/(2*σ_hat_1_squared) + 1/(2*σ_hat_2_squared) ) + 2 * n * (F_n^2) / (σ_hat_1_squared/4 + σ_hat_2_squared/4)

    return T_n
end








In [ ]:
function test_statistic_energy(pms_1::Vector{Normal}, pms_2::Vector{Normal})
    # Recall that we assume that each of the sample is Normal distribution, and for W_2 metric we only need their means.
    n = length(pms_1)
    @assert n == length(pms_2) "two samples of probability measures must have same size "

    X = [pms_1[i].μ for i in 1:n] # collect all the means from first samples of probability measures
    Y = [pms_2[i].μ for i in 1:n] # collect all the means from second samples of probability measures

    distances_X = [abs(X[i] - X[j]) for i in 1:n, j in 1:n]
    distances_Y = [abs(Y[i] - Y[j]) for i in 1:n, j in 1:n]
    distances_XY = [abs(X[i] - Y[j]) for i in 1:n, j in 1:n]


    distance = 2 * mean(distances_XY) - mean(distances_X) - mean(distances_Y)
    T_n = distance * n / 2
    return T_n
end



function decide_energy_boostrap(pms_1::Vector{Normal}, pms_2::Vector{Normal}, n_boostrap::Int, θ = 0.05)
    # given two samples of probobability measures, this function returns 0 or 1, i.e. either rejects or accepts 
    # null hypothesis for given significance level θ according to Energy statistic (Szekely & Rizzo 2004).

    # n_boostrap: number of times we repeat bootstrap procedure to estimate the quantile of the test statistic

    n_top = length(pms_1)
    T_n = test_statistic_energy(pms_1, pms_2) # test statistic
    # obtain quantile using bootstrap approach
    T_n_boostrap = zeros(n_boostrap)
    for s in 1:n_boostrap
        allmeasures = vcat(pms_1, pms_2) # collect all probability measures into one vector
        pms_1_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        pms_2_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        T_n_boostrap[s] = test_statistic_energy(pms_1_boostrap, pms_2_boostrap) # test statistic from boostraped sample
    end
    threshold = quantile(T_n_boostrap, 1-θ)
    decision = 1.0*(T_n > threshold) # 1.0 if T_n > threshold, 0.0 otherwise.
    return decision
end

In [ ]:
function decide_dm_asympt(pms_1::Vector{Normal}, pms_2::Vector{Normal}, n_boostrap::Int, θ = 0.05)
    # given two samples of probobability measures, this function returns 0 or 1, i.e. either rejects or accepts 
    # null hypothesis for given significance level θ according to Dubey & Muller.

    # threshold is choosen from asymptotic distribution of test statistic which is chi-squared.

    n_top = length(pms_1)
    T_n = test_statistic_dm(pms_1, pms_2) # test statistic
    
    threshold = quantile(Chisq(1), 1 - θ) # obtain quantile from asymptotic distribution of test statistic
    decision = 1.0*(T_n > threshold) # 1.0 if T_n > threshold, 0.0 otherwise.
    return decision
end

function decide_dm_boostrap(pms_1::Vector{Normal}, pms_2::Vector{Normal}, n_boostrap::Int, θ = 0.05)
    # given two samples of probobability measures, this function returns 0 or 1, i.e. either rejects or accepts 
    # null hypothesis for given significance level θ according to Dubey & Muller.

    # n_boostrap: number of times we repeat bootstrap procedure to estimate the quantile of the test statistic

    n_top = length(pms_1)
    T_n = test_statistic_dm(pms_1, pms_2) # test statistic
    # obtain quantile using bootstrap approach
    T_n_boostrap = zeros(n_boostrap)
    for s in 1:n_boostrap
        allmeasures = vcat(pms_1, pms_2) # collect all probability measures into one vector
        pms_1_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        pms_2_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        T_n_boostrap[s] = test_statistic_dm(pms_1_boostrap, pms_2_boostrap) # test statistic from boostraped sample
    end
    threshold = quantile(T_n_boostrap, 1-θ)
    decision = 1.0*(T_n > threshold) # 1.0 if T_n > threshold, 0.0 otherwise.
    return decision
end



function decide_dm_boostrap(pms_1::Vector{Float64}, pms_2::Vector{Float64}, n_boostrap::Int, θ = 0.05)
    # given two samples of means of normal distributions with variance 1,
    # this function returns 0 or 1, i.e. either rejects or accepts 
    # null hypothesis for given significance level θ according to Dubey & Muller.

    # n_boostrap: number of times we repeat bootstrap procedure to estimate the quantile of the test statistic

    n_top = length(pms_1)
    T_n = test_statistic_dm(pms_1, pms_2) # test statistic
    # obtain quantile using bootstrap approach
    T_n_boostrap = zeros(n_boostrap)
    for s in 1:n_boostrap
        allmeasures = vcat(pms_1, pms_2) # collect all probability measures into one vector
        pms_1_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        pms_2_boostrap = sample(allmeasures, n_top; replace=true) # resample from pooled probability measures
        T_n_boostrap[s] = test_statistic_dm(pms_1_boostrap, pms_2_boostrap) # test statistic from boostraped sample
    end
    threshold = quantile(T_n_boostrap, 1-θ)
    decision = 1.0*(T_n > threshold) # 1.0 if T_n > threshold, 0.0 otherwise.
    return decision
end



In [ ]:
function get_thresholds_permutation_hipm_wow(q_1::PPM, q_2::PPM, n_top::Int, n_bottom::Int, n_permutations::Int, θ::Float64)
    # given laws of RPMs q_1, q_2, approximates the quantile of d(Q_{n,m}^1, Q_{n,m}^2) for both distance functions hipm,wow, using
    # permutation approach. 
    permuted_samples_dlip = zeros(n_permutations) # store samples of distances
    permuted_samples_ww = zeros(n_permutations) # store samples of distances

    hier_sample_1, hier_sample_2 = generate_emp(q_1, n_top, n_bottom), generate_emp(q_2, n_top, n_bottom)
    for k in 1:n_permutations
        total_rows = vcat(hier_sample_1.atoms, hier_sample_2.atoms) # collect all rows
        random_indices = randperm(2*n_top) # indices to distribute rows to new hierarchical meausures

        atoms_1 = total_rows[random_indices[1:n_top],:] # first rows indexed by n_top random indices to the atoms_1
        atoms_2 = total_rows[random_indices[n_top+1:end],:] # first rows indexed by n_top random indices to the atoms_2

        # now we need to make sure that we get the proper bounds on the all observations from hierarchical samples
        a_1 = minimum(atoms_1) # left end of an interaval where observations in hier_sample_1 take values
        b_1 = maximum(atoms_1) # right end of an interaval where observations in hier_sample_1 take values

        a_2 = minimum(atoms_2) # left end of an interaval where in hier_sample_2 observations take values
        b_2 = maximum(atoms_2) # right end of an interaval where in hier_sample_2 observations take values

        a,b = minimum([a_1,a_2]), maximum([b_1,b_2])
        hier_sample_1_permuted = emp_ppm(atoms_1, n_top, n_bottom, a, b)
        hier_sample_2_permuted = emp_ppm(atoms_2, n_top, n_bottom, a, b)

        
        permuted_samples_dlip[k] = dlip(hier_sample_1_permuted, hier_sample_2_permuted)
        permuted_samples_ww[k] = ww(hier_sample_1_permuted, hier_sample_2_permuted)
    end

    threshold_hipm = quantile(permuted_samples_dlip, 1 - θ)
    threshold_wow = quantile(permuted_samples_ww, 1 - θ)

    return threshold_hipm, threshold_wow
end
    





function get_thresholds_boostrap_hipm_wow(q_1::PPM, q_2::PPM, n_top::Int, n_bottom::Int, n_boostrap::Int, θ::Float64)
    # given laws of RPMs q_1, q_2, approximates the quantile of d(Q_{n,m}^1, Q_{n,m}^2) for both distance functions hipm,wow, using
    # permutation approach. 
    permuted_samples_dlip = zeros(n_boostrap) # store samples of distances
    permuted_samples_ww = zeros(n_boostrap) # store samples of distances

    hier_sample_1, hier_sample_2 = generate_emp(q_1, n_top, n_bottom), generate_emp(q_2, n_top, n_bottom)
    for k in 1:n_boostrap
        total_rows = vcat(hier_sample_1.atoms, hier_sample_2.atoms) # collect all rows
        
        indices_1 = sample(1:2*n_top, n_top; replace = true)
        indices_2 = sample(1:2*n_top, n_top; replace = true)
        atoms_1 = total_rows[indices_1,:]  # resample from pooled hierarchical sample
        atoms_2 = total_rows[indices_2,:]  # resample from pooled hierarchical sample
        
        # now we need to make sure that we get the proper bounds on the all observations from hierarchical samples
        a_1 = minimum(atoms_1) # left end of an interaval where observations in hier_sample_1 take values
        b_1 = maximum(atoms_1) # right end of an interaval where observations in hier_sample_1 take values

        a_2 = minimum(atoms_2) # left end of an interaval where in hier_sample_2 observations take values
        b_2 = maximum(atoms_2) # right end of an interaval where in hier_sample_2 observations take values

        a,b = minimum([a_1,a_2]), maximum([b_1,b_2])
        hier_sample_1_permuted = emp_ppm(atoms_1, n_top, n_bottom, a, b)
        hier_sample_2_permuted = emp_ppm(atoms_2, n_top, n_bottom, a, b)

        
        permuted_samples_dlip[k] = dlip(hier_sample_1_permuted, hier_sample_2_permuted)
        permuted_samples_ww[k] = ww(hier_sample_1_permuted, hier_sample_2_permuted)
    end

    threshold_hipm = quantile(permuted_samples_dlip, 1 - θ)
    threshold_wow = quantile(permuted_samples_ww, 1 - θ)

    return threshold_hipm, threshold_wow
end
    



In [ ]:
using RCall


function denanova_from_r(mu_1::Vector{Float64}, mu_2::Vector{Float64};
                           sd::Float64 = 1.0, nq::Int = 51, qmin=0.01, qmax=0.99,
                           n_boostrap::Int = 1000, seed = 1234, theta::Float64 = 0.05)

    @rput mu_1 mu_2 sd nq qmin qmax n_boostrap seed

    R"""
    # if (!requireNamespace("frechet", quietly = TRUE)) {
    #   install.packages("frechet", repos="https://cloud.r-project.org")
    # }

    set.seed(seed)

    n1 <- length(mu_1)
    n2 <- length(mu_2)
    qSup <- seq(qmin, qmax, length.out = nq)

    Y1 <- lapply(seq_len(n1), function(i) qnorm(qSup, mean = mu_1[i], sd = sd))
    Y2 <- lapply(seq_len(n2), function(i) qnorm(qSup, mean = mu_2[i], sd = sd))

    Ly <- c(Y1, Y2)
    Lx <- qSup
    group <- c(rep(1, n1), rep(2, n2))

    res <- frechet::DenANOVA(qin = Ly, supin = Lx, group = group,
                    optns = list(boot = TRUE, R = n_boostrap))

    p_asym <- res$pvalAsy; p_boot <- res$pvalBoot
    """
    @rget p_asym p_boot
    return 1 * (p_boot < theta)
end


function rej_rates_denanova_my(q_1::PPM, q_2::PPM, S::Int, n_top::Int, n_boostrap::Int, θ::Float64)
    # obtain rejection rates of denanova and my implementation of dm with boostrap
    # for given laws of RPMs q_1 and q_2
    # S : number of times we simulate two samples from given laws of RPM
    # n_top : number of probability measures we simulate from each q
    # n_boostrap : number of times we repeat bootstrap procedure to estimate the quantile of the test statistic.
    
    rej_rate_denanova, rej_rate_my = 0.0, 0.0

    println("law of RPMS are: $(q_1)")
    println("$(q_2)")

    for s in 1:S
        mu_1, mu_2 = [pm_1.μ for pm_1 in generate_prob_measures(q_1, n_top)], [pm_2.μ for pm_2 in generate_prob_measures(q_2, n_top)]
        println("s is $(s)")
        rej_rate_denanova += denanova_from_r(mu_1, mu_2; n_boostrap=100, theta = θ)
        rej_rate_my += decide_dm_boostrap(mu_1, mu_2, n_boostrap, θ)
    end
    rej_rate_denanova /= S
    rej_rate_my /= S
    return rej_rate_denanova, rej_rate_my
end


In [ ]:
function rejection_rate(q_1::PPM, q_2::PPM, S::Int, n_top::Int, n_bottom::Int, n_boostrap::Int, θ::Float64, boostrap::Bool=true)
    # Given two laws of RPM, q_1 and q_2, estimate the rejection rate for given significance level θ for 3 testing schemes:
    # Dubey & Mueller, HIPM, WoW, Energy statistic

    # S : number of times we simulate two samples from given laws of RPM
    # n_top : number of probability measures we simulate from each q
    # n_bottom : number of random variables we simulate from each of the probability measure from q
    # n_boostrap : number of times we repeat bootstrap procedure to estimate the quantile of the test statistic. Note that this is 
    #              same as number of permutations for HIPM and WoW.
    
    rej_rate_dm, rej_rate_hipm, rej_rate_wow, rej_rate_energy = 0.0, 0.0, 0.0, 0.0

    # Instead of getting the threshold for HIPM and WoW based on hierarchical sample everytime, for fixed q_1 and q_2, we obtain
    # threshold once from some generated hierarchical samples and then use it for other hierarchical samples.
    if boostrap
        threshold_hipm, threshold_wow = get_thresholds_boostrap_hipm_wow(q_1, q_2, n_top, n_bottom, n_boostrap, θ) 
    else
        threshold_hipm, threshold_wow = get_thresholds_permutation_hipm_wow(q_1, q_2, n_top, n_bottom, n_boostrap, θ) 
    end

    println("law of RPMS are: $(q_1)")
    println("$(q_2)")

    for s in 1:S
        println("S is $(s)")
        pms_1, pms_2 = generate_prob_measures(q_1, n_top), generate_prob_measures(q_2, n_top) # generate n_top probability measures
                                            # from q_1 and q_2

        mu_1, mu_2 = [pm_1.μ for pm_1 in pms_1], [pm_2.μ for pm_2 in pms_2] # collect means of all probability measures in pms_1 and pms_2

        hier_sample_1, hier_sample_2 = generate_emp(pms_1, n_top, n_bottom), generate_emp(pms_2, n_top, n_bottom) # generate n_bottom
                                            # random variables from each probability measures in pms_1 and pms_2
        # endpoints of the sample space for observatinos from hier_sample_1 and hier_sample_2 might be different, so we fix it
        a = minimum([minimum(hier_sample_1.atoms), minimum(hier_sample_2.atoms)])
        b = maximum([maximum(hier_sample_1.atoms), maximum(hier_sample_2.atoms)])
        hier_sample_1.a = a
        hier_sample_1.b = b
        hier_sample_2.a = a
        hier_sample_2.b = b

        # record if testing schemes reject
        rej_rate_dm += denanova_from_r(mu_1, mu_2; n_boostrap=n_boostrap, theta = θ)  
        rej_rate_hipm += 1*(dlip(hier_sample_1, hier_sample_2) > threshold_hipm)
        rej_rate_wow += 1*(ww(hier_sample_1, hier_sample_2) > threshold_wow)

        rej_rate_energy += decide_energy_boostrap(pms_1, pms_2, n_boostrap, θ)
    end
    rej_rate_dm /= S
    rej_rate_hipm /= S
    rej_rate_wow /= S
    rej_rate_energy /= S
    return rej_rate_dm,  rej_rate_hipm, rej_rate_wow, rej_rate_energy
end


In [ ]:
function power_per_parameter(δs::Vector{Float64}, power::Function)
    # given δs and power function, computes power function per each δ and returns rejection rates per each δ for each
    # testing scheme: DM, HIPM, WoW, Energy statistic,
    rej_rates = zeros(length(δs), 4) # per each delta and testing scheme

    for i in 1:length(δs)
        rej_rates[i,:] .= power(δs[i]) # returns rejection rate for δ[i] for each testing scheme
    end
    return rej_rates
end

# Figure 1, Left, of the paper

Now we plot the power function per $\delta$ as it is in their paper for 


We let $ n = 35 $

In [ ]:
n_top, n_bottom = 5,2
n_boostrap = 5
S = 20
θ = 0.05

Random.seed!(1234) # for reproducibility

# maybe variance should be sqrt(0.5) instead of 0.5 ???
power_tnormal_normal(δ) = rejection_rate(tnormal_normal(0.0, 0.5, -10.0, 10.0), tnormal_normal(δ, 0.5, -10.0, 10.0),S, n_top, n_bottom, n_boostrap, θ, true) # powers per δ for each testing scheme

δs = collect(-1.0:0.1:1.0)
rejection_rates_tnormal_normal = power_per_parameter(δs, power_tnormal_normal)
plot_power = plot(title = "Power of 4 testing schemes", xlabel = "δ", ylabel = "Power", xlims=(-1.0, 1.1), ylims = (-0.1, 1.1))
plot!(plot_power, δs, rejection_rates_tnormal_normal[:,1], label = "dm", color = "red")
plot!(plot_power, δs, rejection_rates_tnormal_normal[:,2], label = "hipm", color = "green")
plot!(plot_power, δs, rejection_rates_tnormal_normal[:,3], label = "wow", color = "brown")
plot!(plot_power, δs, rejection_rates_tnormal_normal[:,4], label = "Energy", color = "blue")
filepath = joinpath(pwd(), "frechet/figure1")
savefig(plot_power,joinpath(filepath, "power_tnormal_normal_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_nboostrap=$(n_boostrap).png"))
# # #minimum_power_dm = minimum(power_function_dm[])
# # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")




Now for $n = 100$

In [ ]:
# n_top, n_bottom = 100,2000
# n_boostrap = 200
# S = 100
# θ = 0.05

# Random.seed!(1234) # for reproducibility

# # maybe variance should be sqrt(0.5) instead of 0.5 ???
# power_tnormal_normal(δ) = rejection_rate(tnormal_normal(0.0, 0.5, -10.0, 10.0), tnormal_normal(δ, 0.5, -10.0, 10.0),S, n_top, n_bottom, n_boostrap, θ,true) # powers per δ for each testing scheme

# δs = collect(-1.0:0.1:1.0)
# rejection_rates_tnormal_normal = power_per_parameter(δs, power_tnormal_normal)
# fig_1 = plot(title = "Power of 4 testing schemes", xlabel = "δ", ylabel = "Power", xlims=(-1.0, 1.1), ylims = (-0.1, 1.1))
# plot!(fig_1, δs, rejection_rates_tnormal_normal[:,1], label = "dm", color = "red")
# plot!(fig_1, δs, rejection_rates_tnormal_normal[:,2], label = "hipm", color = "green")
# plot!(fig_1, δs, rejection_rates_tnormal_normal[:,3], label = "wow", color = "brown")
# plot!(fig_1, δs, rejection_rates_tnormal_normal[:,4], label = "Energy", color = "blue")
# filepath = joinpath(pwd(), "frechet/figure1")
# savefig(fig_1,joinpath(filepath, "power_tnormal_normal_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_nboostrap=$(n_boostrap).png"))

# # # #minimum_power_dm = minimum(power_function_dm[])
# # # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")




# Figure 1, Right of the paper

In [ ]:
# n_top, n_bottom = 100,2000
# n_boostrap = 100
# S = 100
# θ = 0.05

# Random.seed!(1234) # for reproducibility

# # maybe variance should be sqrt(0.5) instead of 0.5 ???
# power_tnormal_normal(τ) = rejection_rate(tnormal_normal(0.0, 0.2, -10.0, 10.0), tnormal_normal(0.0, 0.2*τ, -10.0, 10.0),S, n_top, n_bottom, n_boostrap, θ, true) # powers per δ for each testing scheme

# τs = collect(0.5:0.25:3.0)
# rejection_rates_tnormal_normal = power_per_parameter(τs, power_tnormal_normal)
# fig_2 = plot(title = "Power of 4 testing schemes", xlabel = "τ", ylabel = "Power", xlims=(0.0, 3.1), ylims = (-0.1, 1.1))
# plot!(fig_2, τs, rejection_rates_tnormal_normal[:,1], label = "dm", color = "red")
# plot!(fig_2, τs, rejection_rates_tnormal_normal[:,2], label = "hipm", color = "green")
# plot!(fig_2, τs, rejection_rates_tnormal_normal[:,3], label = "wow", color = "brown")
# plot!(fig_2, τs, rejection_rates_tnormal_normal[:,4], label = "Energy", color = "blue")
# filepath = joinpath(pwd(), "frechet/figure1")
# savefig(fig_2,joinpath(filepath, "power_tnormal_normal_varying_variance_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_nboostrap=$(n_boostrap).png"))

# # # #minimum_power_dm = minimum(power_function_dm[])
# # # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")




# Example where DM should fail

Now let us do same simulations on different law of random probability measure: 

$\mu \sim Normal(\delta, \sigma)$ and define $ P = Normal(\mu, 1).$

In this case, if I have $Q^1,Q^2$ where they generate means from $Normal(0.0, 1.0), Normal(0.0,1.5)$, then they are different but Frechet means are same. So power should be low

With Boostrap

In [ ]:
# n_top, n_bottom = 700,200
# n_boostrap = 40
# S = 40
# θ = 0.05

# Random.seed!(1234) # for reproducibility

# a, b = 0.0, 2.0
# μ, σ = 1.0, sqrt(1/3)

# mixture(λ) = mixture_ppm(normal_normal(μ,σ), uniform_normal(a,b), λ) # low lambda means more to second distribution which here is uniform normal
# power_normal_uniform(λ) = rejection_rate(normal_normal(μ,σ), mixture(λ), S, n_top, n_bottom, n_boostrap, θ, true) 

# λs = collect(0.0:0.25:1.0)
# rejection_rates_normal_uniform = power_per_parameter(λs, power_normal_uniform)

# # # #minimum_power_dm = minimum(power_function_dm[])
# # # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")




In [ ]:
# fig_counter_example = plot(title = "Power of 4 testing schemes", xlabel = "λ", ylabel = "Power", xlims=(-0.1, 1.1), ylims = (-0.1, 1.1))
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,1], label = "dm", color = "red")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,2], label = "hipm", color = "green")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,3], label = "wow", color = "brown")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,4], label = "Energy", color = "blue")


In [ ]:
# fig_counter_example = plot(title = "Power of 4 testing schemes", xlabel = "λ", ylabel = "Power", xlims=(-0.1, 1.1), ylims = (-0.1, 1.1))
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,1], label = "dm", color = "red")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,2], label = "hipm", color = "green")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,3], label = "wow", color = "brown")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,4], label = "Energy", color = "blue")

# filepath = joinpath(pwd(), "frechet/counterexample")
# savefig(fig_counter_example,joinpath(filepath, "power_normal_uniform_counter_example_boostrap_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_nboostrap=$(n_boostrap).png"))

# With permutation

In [ ]:
# n_top, n_bottom = 10,2000
# n_permutations = 7
# S = 10
# θ = 0.05

# Random.seed!(1234) # for reproducibility

# a, b = 0.0, 2.0
# μ, σ = 1.0, sqrt(1/3)

# mixture(λ) = mixture_ppm(normal_normal(μ,σ), uniform_normal(a,b), λ) # low lambda means more to second distribution which here is uniform normal
# power_normal_uniform(λ) = rejection_rate(normal_normal(μ,σ), mixture(λ), S, n_top, n_bottom, n_boostrap, θ, false) 

# λs = collect(0.0:0.25:1.0)
# rejection_rates_normal_uniform = power_per_parameter(λs, power_normal_uniform)

# # # #minimum_power_dm = minimum(power_function_dm[])
# # # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")




In [ ]:
# fig_counter_example = plot(title = "Power of 4 testing schemes", xlabel = "λ", ylabel = "Power", xlims=(-0.1, 1.1), ylims = (-0.1, 1.1))
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,1], label = "dm", color = "red")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,2], label = "hipm", color = "green")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,3], label = "wow", color = "brown")
# scatter!(fig_counter_example, λs, rejection_rates_normal_uniform[:,4], label = "Energy", color = "blue")


In [ ]:
# fig_counter_example = plot(title = "Power of 4 testing schemes", xlabel = "λ", ylabel = "Power", xlims=(-0.1, 1.1), ylims = (-0.1, 1.1))
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,1], label = "dm", color = "red")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,2], label = "hipm", color = "green")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,3], label = "wow", color = "brown")
# plot!(fig_counter_example, λs, rejection_rates_normal_uniform[:,4], label = "Energy", color = "blue")


# filepath = joinpath(pwd(), "frechet/counterexample")
# savefig(fig_counter_example,joinpath(filepath, "power_normal_uniform_counter_example_permutation_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_permutations=$(n_permutations).png"))

# Let us test if my code is same as the code written in R

In [ ]:





n_top, n_bottom = 100,2000
n_boostrap = 200
S = 200
θ = 0.05

Random.seed!(1234) # for reproducibility
δs = collect(-1.0:0.05:1.0)
rej_rate_denanova_per_delta, rej_rate_my_per_delta = zeros(length(δs)), zeros(length(δs))
for (i, δ) in enumerate(δs)
    q_1, q_2 = tnormal_normal(0.0, 0.5, -10.0, 10.0), tnormal_normal(δ, 0.5, -10.0, 10.0)

    rej_rate_denanova, rej_rate_my = rej_rates_denanova_my(q_1, q_2, S, n_top, n_boostrap, θ)
    rej_rate_denanova_per_delta[i] = rej_rate_denanova
    rej_rate_my_per_delta[i] = rej_rate_my
end
fig_denanova_vs_my = plot(title = "Power of DenANOVA vs my implementation of DM", xlabel = "δ", ylabel = "Power", xlims=(-1.1, 1.1), ylims = (-0.1, 1.1))
plot!(fig_denanova_vs_my, δs, rej_rate_denanova_per_delta, label = "DenANOVA", color = "red")
plot!(fig_denanova_vs_my, δs, rej_rate_my_per_delta, label = "My DM", color = "blue")

# # maybe variance should be sqrt(0.5) instead of 0.5 ???
# power_tnormal_normal(δ) = rejection_rate(tnormal_normal(0.0, 0.5, -10.0, 10.0), tnormal_normal(δ, 0.5, -10.0, 10.0),S, n_top, n_bottom, n_boostrap, θ, true) # powers per δ for each testing scheme

# δs = collect(-1.0:0.1:1.0)
# rejection_rates_tnormal_normal = power_per_parameter(δs, power_tnormal_normal)
# plot_power = plot(title = "Power of 4 testing schemes", xlabel = "δ", ylabel = "Power", xlims=(-1.0, 1.1), ylims = (-0.1, 1.1))
# plot!(plot_power, δs, rejection_rates_tnormal_normal[:,1], label = "dm", color = "red")
# plot!(plot_power, δs, rejection_rates_tnormal_normal[:,2], label = "hipm", color = "green")
# plot!(plot_power, δs, rejection_rates_tnormal_normal[:,3], label = "wow", color = "brown")
# plot!(plot_power, δs, rejection_rates_tnormal_normal[:,4], label = "Energy", color = "blue")
# filepath = joinpath(pwd(), "frechet/figure1")
# savefig(plot_power,joinpath(filepath, "power_tnormal_normal_n_top=$(n_top)_n_bottom=$(n_bottom)_S=$(S)_nboostrap=$(n_boostrap).png"))
# # # #minimum_power_dm = minimum(power_function_dm[])
# # # #hline!([minimum_power_dm], color=:red, linestyle=:dash, label="minimum power for dm")







In [ ]:
rej_rate_denanova_per_delta

In [ ]:
differences = abs.(rej_rate_my_per_delta .- rej_rate_denanova_per_delta)

In [ ]:
plot(δs, differences, title = "Difference between DenANOVA and my implementation of DM", xlabel = "δ",ylabel = "Absolute difference in power", xlims=(-1.1, 1.1), ylims = (-0.05, 0.2), label = "Absolute difference", color = "purple")